# Counterfactuals guided by prototypes on Boston housing dataset

This notebook goes through an example of [prototypical counterfactuals](../methods/CFProto.ipynb) using [k-d trees](https://en.wikipedia.org/wiki/K-d_tree) to build the prototypes. Please check out [this notebook](./cfproto_mnist.ipynb) for a more in-depth application of the method on MNIST using (auto-)encoders and trust scores.

In this example, we will train a simple neural net to predict whether house prices in the Boston area are above the median value or not. We can then find a counterfactual to see which variables need to be changed to increase or decrease a house price above or below the median value.

In [1]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs 
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
from sklearn.datasets import load_boston
from alibi.explainers.cfproto import CounterfactualProto
from sklearn.preprocessing import StandardScaler, MinMaxScaler

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.5.1
Eager execution enabled:  False


## Load and prepare GMSC dataset

In [2]:
def load_GMSC(filename):

    df = pd.read_csv(filename)
    df = df.drop("Unnamed: 0", axis=1) # drop id column
    df = df.loc[df["DebtRatio"] <= df["DebtRatio"].quantile(0.975)]
    df = df.loc[(df["RevolvingUtilizationOfUnsecuredLines"] >= 0) & (df["RevolvingUtilizationOfUnsecuredLines"] < 13)]
    df = df.loc[df["NumberOfTimes90DaysLate"] <= 17] 
    dependents_mode = df["NumberOfDependents"].mode()[0] # impute with mode
    df["NumberOfDependents"] = df["NumberOfDependents"].fillna(dependents_mode)
    income_median = df["MonthlyIncome"].median()
    df["MonthlyIncome"] = df["MonthlyIncome"].fillna(income_median)
    mean = df["MonthlyIncome"].mean()
    std = df["MonthlyIncome"].std()
    df.loc[df["MonthlyIncome"].isnull()]["MonthlyIncome"] = np.random.normal(loc=mean, scale=std, size=len(df.loc[df["MonthlyIncome"].isnull()]))

    y = df['SeriousDlqin2yrs']
    idx1 = np.argwhere(y.values == 0).squeeze()
    idx2 = np.argwhere(y.values == 1).squeeze()
    idx3 = idx1[0:len(idx2)]
    idx4 = np.concatenate((idx2, idx3))
    np.random.seed(0)
    np.random.shuffle(idx4)
    idx5 = list(set(np.arange(len(df))).difference(idx4))

    data1 = df.iloc[idx4]
    data2 = df.iloc[idx5]

    return data1, data2


In [3]:
def train_test_split(data):

    x = data.iloc[:, 1:]
    y = data.iloc[:, 0]

    x_ = x.to_numpy().astype(np.float32)
    y_ = y.to_numpy().astype(np.float32)
    y_ = y_[:, np.newaxis]

    x_train, y_train = x_[0: int(len(x_) * 0.5)], y_[0: int(len(y_) * 0.5)]
    x_query, y_query = x_[int(len(x_) * 0.5): int(len(x_) * 0.75)], y_[int(len(y_) * 0.5):int(len(y_) * 0.75)]
    x_test, y_test = x_[int(len(x_) * 0.75): ], y_[int(len(y_) * 0.75):]

    return x_train, y_train, x_query, y_query, x_test, y_test

In [5]:
data1, data2 = load_GMSC('../cs-training.csv')
train_x, train_y, query_x, query_y, test_x, test_y = train_test_split(data1)
scaler = StandardScaler()
strain_x = scaler.fit_transform(train_x)
squery_x = scaler.transform(query_x)
stest_x = scaler.transform(test_x)

Standardize data

Define train and test set

In [6]:
otrain_y = to_categorical(train_y)
oquery_y = to_categorical(query_y)
otest_y = to_categorical(test_y)

## Train model

In [7]:
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
def nn_model():
    x_in = Input(shape=(10,))
    x = Dense(20, activation='relu')(x_in)
    x = Dense(10, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [9]:
nn = nn_model()
nn.summary()
nn.fit(strain_x, otrain_y, batch_size=64, epochs=500, verbose=0)
nn.save('nn_GMSC.h5', save_format='h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 20)                220       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 452
Trainable params: 452
Non-trainable params: 0
_________________________________________________________________


In [10]:
nn = load_model('nn_GMSC.h5')
print(nn.evaluate(squery_x, oquery_y))
print(nn.evaluate(stest_x, otest_y))

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


[0.4701878186175948, 0.7844971]
[0.49762366521687906, 0.76854354]


# Train Autoencoder

In [ ]:
def ae_model():
    # encoder
    x_in = Input(shape=(12,))
    x = Dense(30, activation='relu')(x_in)
    x = Dense(15, activation='relu')(x)
    encoded = Dense(5, activation=None)(x)
    encoder = Model(x_in, encoded)

    # decoder
    dec_in = Input(shape=(5,))
    x = Dense(15, activation='relu')(dec_in)
    x = Dense(30, activation='relu')(x)
    decoded = Dense(12, activation=None)(x)
    decoder = Model(dec_in, decoded)

    # autoencoder = encoder + decoder
    x_out = decoder(encoder(x_in))
    autoencoder = Model(x_in, x_out)
    autoencoder.compile(optimizer='adam', loss='mse')

    return autoencoder, encoder, decoder

In [ ]:
ae, enc, dec = ae_model()
ae.summary()
ae.fit(strain_x, strain_x, batch_size=128, epochs=100, validation_data=(strain_x, strain_x), verbose=1)

In [ ]:
ae.save('boston_ae.h5', save_format='h5')
enc.save('boston_enc.h5', save_format='h5')

## Generate counterfactual guided by the nearest class prototype

Original instance:

In [ ]:
X = squery_x[1].reshape((1,) + squery_x[1].shape)
shape = X.shape

Run counterfactual:

In [ ]:
# define model
nn = load_model('nn_boston.h5')

# initialize explainer, fit and generate counterfactual
cf = CounterfactualProto(nn, shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=(strain_x.min(axis=0), strain_x.max(axis=0)), 
                         c_init=1., c_steps=10)

cf.fit(strain_x)
explanation = cf.explain(X)

The prediction flipped from 0 (value below the median) to 1 (above the median):

In [ ]:
print(f'Original prediction: {explanation.orig_class}')
print('Counterfactual prediction: {}'.format(explanation.cf['class']))

Let's take a look at the counterfactual. To make the results more interpretable, we will first undo the pre-processing step and then check where the counterfactual differs from the original instance:

In [ ]:
orig = scaler.inverse_transform(X)
counterfactual = scaler.inverse_transform(explanation.cf['X'])
delta = counterfactual - orig
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-4:
        print('{}: {}'.format(f, delta[0][i]))

So in order to increase the house price, the proportion of owner-occupied units built prior to 1940 should decrease by ~11-12%. This is not surprising since the proportion for the observation is very high at 93.6%. Furthermore, the % of the population with "lower status" should decrease by ~5%.

In [ ]:
print('% owner-occupied units built prior to 1940: {}'.format(orig[0][5]))
print('% lower status of the population: {}'.format(orig[0][11]))

In [ ]:
print(explanation.cf)

Generate the counterfactual explanation for all query instances.

In [ ]:
query_cf = np.zeros_like(squery_x)
query_cf_y = np.zeros(len(squery_x))
for idx in range(len(squery_x)):
    print(idx)
    X = squery_x[idx].reshape((1,) + squery_x[idx].shape)
    explanation = cf.explain(X)
    query_cf[idx] = explanation.cf['X']
    query_cf_y[idx] = explanation.cf['class']

In [ ]:
tmp = np.concatenate((scaler.inverse_transform(query_cf), query_cf_y[:, np.newaxis]), axis = 1)
np.save("boston_housing_query_cf.npy", tmp)

In [ ]:
query_ccf = np.zeros_like(squery_x)
query_ccf_y = np.zeros(len(squery_x))

for idx in range(len(query_cf)):
    print(idx)
    X = query_cf[idx].reshape((1,) + query_cf[idx].shape)
    explanation = cf.explain(X)
    query_ccf[idx] = explanation.cf['X']
    query_ccf_y[idx] = explanation.cf['class']

In [ ]:
tmp1 = np.concatenate((scaler.inverse_transform(query_ccf), query_pnss_y[:, np.newaxis]), axis = 1)
np.save("boston_housing_query_2cf.npy", tmp1)

In [ ]:
print(query_pnss_y)

In [ ]:
print(query_pns_y)

In [ ]:
print(query_pnss[0].max())

In [ ]:
print(query_pns - query_pnss)